In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('training_v2.csv')
test = pd.read_csv('unlabeled.csv')
wid_dict = pd.read_csv('WiDS Datathon 2020 Dictionary.csv')
sol_temp = pd.read_csv('solution_template.csv')
sam = pd.read_csv('samplesubmission.csv')

In [ ]:
train.select_dtypes(exclude=['object'])

In [ ]:
train.info()

In [ ]:
pd.set_option('display.max_colwidth', 500)
#pd.set_option('display.max_width', 1000)
wid_dict

In [ ]:
sam.head()

In [ ]:
train.icu_stay_type.unique()

In [ ]:
for i in train.columns:
    print(train[i].unique())

In [ ]:
train.isnull().sum()

In [ ]:
train.isnull().sum()/len(train)*100

In [ ]:
all_data_na = train.isnull().sum()/len(train)*100

In [ ]:
all_data_na.sort_values(ascending=False, inplace=True)

In [ ]:
all_data_na.head(5)

In [ ]:
all_data_na.index

In [ ]:
target=all_data_na.hospital_death
all_data_na.drop('hospital_death', inplace=True)

all_data_na.drop(all_data_na[all_data_na > 10].index, inplace=True) # drop rows with percents greater than 10
percent_null = pd.DataFrame({'null_cols': all_data_na, 'dataTypes': train[all_data_na.index].dtypes})
#percent_null

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
cat_var = encoder.fit_transform(train['Accident & Emergency'])
cat_var

#to get the order of arrangement,
print(encoder.classes_)

from sklearn.preprocessing import OneHotEncoder

Oencoder = OneHotEncoder()
one_var = Oencoder.fit_transform(cat_var.reshape(-1, 1))
one_var


'''We can apply both transformations (from text categories to integer categories, then
from integer categories to one-hot vectors) in one shot using the LabelBinarizer
class:'''

from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
cat_var_hot = lb.fit_transform(train['Accident & Emergency'])
cat_var_hot

In [ ]:
Custom Transformers

from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', Imputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
'''how can you join these transformations into a single
pipeline? Scikit-Learn provides a FeatureUnion class for this. You give it a list of
transformers (which can be entire transformer pipelines), and when its transform()
method is called it runs each transformer’s transform() method in parallel, waits for
their output, and then concatenates them and returns the result (and of course calling
its fit() method calls all each transformer’s fit() method). A full pipeline handling
both numerical and categorical attributes may look like this:'''


from sklearn.pipeline import FeatureUnion
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', Imputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])
cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('label_binarizer', LabelBinarizer()),
])
full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

And you can run the whole pipeline simply:
>>> housing_prepared = full_pipeline.fit_transform(housing)
>>> housing_prepared


from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
        

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)

Let’s look at the results:
    
>>> def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

>>> display_scores(tree_rmse_scores)

In [ ]:
to save your model instead of using pickle

from sklearn.externals import joblib

joblib.dump(my_model, "my_model.pkl")
# and later...
my_model_loaded = joblib.load("my_model.pkl")

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]
forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error')
grid_search.fit(housing_prepared, housing_labels)

When you have no idea what value a hyperparameter should have,
a simple approach is to try out consecutive powers of 10 (or a
smaller number if you want a more fine-grained search, as shown
in this example with the n_estimators hyperparameter).

grid_search.best_params_
# gives the best max_features and n_estimators

You can also get the best estimator directly:
>>> grid_search.best_estimator_

And of course the evaluation scores are also available:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
    
    
The grid search approach is fine when you are exploring relatively few combinations,
like in the previous example, but when the hyperparameter search space is large, it is
often preferable to use RandomizedSearchCV instead. This class can be used in much
the same way as the GridSearchCV class, but instead of trying out all possible combinations,
it evaluates a given number of random combinations by selecting a random
value for each hyperparameter at every iteration. This approach has two main benefits:
• If you let the randomized search run for, say, 1,000 iterations, this approach will
explore 1,000 different values for each hyperparameter (instead of just a few values
per hyperparameter with the grid search approach).
• You have more control over the computing budget you want to allocate to hyperparameter
search, simply by setting the number of iterations.

In [ ]:
train1 = train[percent_null.index]

test1 =test[percent_null.index] 

In [ ]:
train1.head()

In [ ]:
en_id = train['encounter_id']
train1.drop(['patient_id', 'hospital_id', 'encounter_id'], axis=1, inplace=True)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# A function to plot a boxplot for each column in train1
for k in train1.keys():
    plt.boxplot(train1[k])
    plt.title('boxplot for column ' + k)
    plt.grid('on')
    plt.show()

In [ ]:
train1.info()

In [ ]:
cat_train = train[['icu_type', 'icu_stay_type', 'icu_admit_source', 'gender', 'apache_2_bodysystem', 'ethnicity', 'apache_3j_bodysystem',]]

In [ ]:
for i in train1.columns:
    if i not in cat_train.columns:
        train1[i].fillna(train[i].median(), inplace = True)

In [ ]:
train1.info()

In [ ]:
def plot_graph(x, y):
    fig, ax = plt.subplots(figsize = (15, 5))
    sns.barplot(x, y)
    plt.xlabel('x', fontSize=13)
    plt.xticks(rotation='90')
    plt.ylabel('Hospital_death', fontSize=13)
    plt.show()

In [ ]:
cat_train.icu_admit_source.unique()

In [ ]:
cat_train.info()

In [ ]:
pd.crosstab(cat_train.icu_admit_source, train.hospital_death, margins=True)

In [ ]:
train1.icu_admit_source = train1.icu_admit_source.fillna('Accident & Emergency')

In [ ]:
test1.icu_admit_source = test1.icu_admit_source.fillna('Accident & Emergency')

In [ ]:
cat_train.gender.unique()

In [ ]:
train1.gender = train1.gender.fillna('Not_Sure')

In [ ]:
cat_train.apache_2_bodysystem.unique()

In [ ]:
train1.apache_2_bodysystem.fillna('Undefined Diagnoses', inplace=True)
train1.apache_2_bodysystem.replace('Undefined diagnoses', 'Undefined Diagnoses', inplace=True)

In [ ]:
cat_train.ethnicity.unique()

In [ ]:
pd.crosstab(cat_train.ethnicity, train.hospital_death, margins=True)

In [ ]:
train1.ethnicity.fillna('Other/Unknown', inplace=True)

In [ ]:
cat_train.apache_3j_bodysystem.unique()

In [ ]:
train1.apache_3j_bodysystem = train1.apache_3j_bodysystem.fillna('Undefined_Diagnoses')

In [ ]:
import seaborn as sns
for i in cat_train.columns:
    plot_graph(train1[i], train['hospital_death'])

In [ ]:
train1['icu_type'][train1['icu_type'] == 'MICU'] = 0
train1['icu_type'][train1['icu_type'] == 'Cardiac ICU'] = 1
train1['icu_type'][train1['icu_type'] == 'Med-Surg ICU'] = 2
train1['icu_type'][train1['icu_type'] == 'Neuro ICU'] = 3
train1['icu_type'][train1['icu_type'] == 'CCU-CTICU'] = 4
train1['icu_type'][train1['icu_type'] == 'SICU'] = 4
train1['icu_type'][train1['icu_type'] == 'CTICU'] = 5
train1['icu_type'][train1['icu_type'] == 'CSICU'] = 6

In [ ]:
train1['icu_stay_type'][train1['icu_stay_type'] == 'transfer'] = 0
train1['icu_stay_type'][train1['icu_stay_type'] == 'readmit'] = 1
train1['icu_stay_type'][train1['icu_stay_type'] == 'admit'] = 2

In [ ]:
train1['icu_admit_source'][train1['icu_admit_source'] == 'Other ICU'] = 0
train1['icu_admit_source'][train1['icu_admit_source'] == 'Other Hospital'] = 1
train1['icu_admit_source'][train1['icu_admit_source'] == 'Floor'] = 2
train1['icu_admit_source'][train1['icu_admit_source'] == 'Accident & Emergency'] = 3
train1['icu_admit_source'][train1['icu_admit_source'] == 'Operating Room / Recovery'] = 4

In [ ]:
train1['apache_2_bodysystem'][train1['apache_2_bodysystem'] == 'Respiratory'] = 0
train1['apache_2_bodysystem'][train1['apache_2_bodysystem'] == 'Cardiovascular'] = 1
train1['apache_2_bodysystem'][train1['apache_2_bodysystem'] == 'Haematologic'] = 2
train1['apache_2_bodysystem'][train1['apache_2_bodysystem'] == 'Undefined Diagnoses'] = 7
train1['apache_2_bodysystem'][train1['apache_2_bodysystem'] == 'Undefined diagnoses'] = 7
train1['apache_2_bodysystem'][train1['apache_2_bodysystem'] == 'Neurologic'] = 3
train1['apache_2_bodysystem'][train1['apache_2_bodysystem'] == 'Trauma'] = 5
train1['apache_2_bodysystem'][train1['apache_2_bodysystem'] == 'Gastrointestinal'] = 4
train1['apache_2_bodysystem'][train1['apache_2_bodysystem'] == 'Renal/Genitourinary'] = 6
train1['apache_2_bodysystem'][train1['apache_2_bodysystem'] == 'Metabolic'] = 8

In [ ]:
train1['apache_3j_bodysystem'][train1['apache_3j_bodysystem'] == 'Sepsis'] = 0
train1['apache_3j_bodysystem'][train1['apache_3j_bodysystem'] == 'Respiratory'] = 1
train1['apache_3j_bodysystem'][train1['apache_3j_bodysystem'] == 'Hematological'] = 2
train1['apache_3j_bodysystem'][train1['apache_3j_bodysystem'] == 'Undefined_Diagnoses'] = 9
train1['apache_3j_bodysystem'][train1['apache_3j_bodysystem'] == 'Neurological'] = 4
train1['apache_3j_bodysystem'][train1['apache_3j_bodysystem'] == 'Cardiovascular'] = 3
train1['apache_3j_bodysystem'][train1['apache_3j_bodysystem'] == 'Gastrointestinal'] = 5
train1['apache_3j_bodysystem'][train1['apache_3j_bodysystem'] == 'Trauma'] = 6
train1['apache_3j_bodysystem'][train1['apache_3j_bodysystem'] == 'Genitourinary'] = 7
train1['apache_3j_bodysystem'][train1['apache_3j_bodysystem'] == 'Musculoskeletal/Skin'] = 8
train1['apache_3j_bodysystem'][train1['apache_3j_bodysystem'] == 'Metabolic'] = 10
train1['apache_3j_bodysystem'][train1['apache_3j_bodysystem'] == 'Gynecological'] = 9

In [ ]:

for i in test1.columns:
    if i not in cat_train.columns:
        test1[i].fillna(test1[i].median(), inplace = True)
        
test1.gender = test1.gender.fillna('Not_Sure')

test1.apache_2_bodysystem.fillna('Undefined Diagnoses', inplace=True)
test1.apache_2_bodysystem.replace('Undefined diagnoses', 'Undefined Diagnoses', inplace=True)

test1.ethnicity.fillna('Other/Unknown', inplace=True)

test1.apache_3j_bodysystem = test1.apache_3j_bodysystem.fillna('Undefined_Diagnoses')

test1['icu_type'][test1['icu_type'] == 'MICU'] = 0
test1['icu_type'][test1['icu_type'] == 'Cardiac ICU'] = 1
test1['icu_type'][test1['icu_type'] == 'Med-Surg ICU'] = 2
test1['icu_type'][test1['icu_type'] == 'Neuro ICU'] = 3
test1['icu_type'][test1['icu_type'] == 'CCU-CTICU'] = 4
test1['icu_type'][test1['icu_type'] == 'SICU'] = 5
test1['icu_type'][test1['icu_type'] == 'CTICU'] = 6
test1['icu_type'][test1['icu_type'] == 'CSICU'] = 7

test1['icu_stay_type'][test1['icu_stay_type'] == 'transfer'] = 0
test1['icu_stay_type'][test1['icu_stay_type'] == 'readmit'] = 1
test1['icu_stay_type'][test1['icu_stay_type'] == 'admit'] = 2


test1['apache_2_bodysystem'][test1['apache_2_bodysystem'] == 'Respiratory'] = 0
test1['apache_2_bodysystem'][test1['apache_2_bodysystem'] == 'Cardiovascular'] = 1
test1['apache_2_bodysystem'][test1['apache_2_bodysystem'] == 'Haematologic'] = 2
test1['apache_2_bodysystem'][test1['apache_2_bodysystem'] == 'Undefined Diagnoses'] = 8
test1['apache_2_bodysystem'][test1['apache_2_bodysystem'] == 'Undefined diagnoses'] = 8
test1['apache_2_bodysystem'][test1['apache_2_bodysystem'] == 'Neurologic'] = 3
test1['apache_2_bodysystem'][test1['apache_2_bodysystem'] == 'Trauma'] = 5
test1['apache_2_bodysystem'][test1['apache_2_bodysystem'] == 'Gastrointestinal'] = 4
test1['apache_2_bodysystem'][test1['apache_2_bodysystem'] == 'Renal/Genitourinary'] = 6
test1['apache_2_bodysystem'][test1['apache_2_bodysystem'] == 'Metabolic'] = 7

test1['apache_3j_bodysystem'][test1['apache_3j_bodysystem'] == 'Sepsis'] = 0
test1['apache_3j_bodysystem'][test1['apache_3j_bodysystem'] == 'Respiratory'] = 1
test1['apache_3j_bodysystem'][test1['apache_3j_bodysystem'] == 'Hematological'] = 2
test1['apache_3j_bodysystem'][test1['apache_3j_bodysystem'] == 'Undefined_Diagnoses'] = 10
test1['apache_3j_bodysystem'][test1['apache_3j_bodysystem'] == 'Neurological'] = 4
test1['apache_3j_bodysystem'][test1['apache_3j_bodysystem'] == 'Cardiovascular'] = 3
test1['apache_3j_bodysystem'][test1['apache_3j_bodysystem'] == 'Gastrointestinal'] = 5
test1['apache_3j_bodysystem'][test1['apache_3j_bodysystem'] == 'Trauma'] = 6
test1['apache_3j_bodysystem'][test1['apache_3j_bodysystem'] == 'Genitourinary'] = 7
test1['apache_3j_bodysystem'][test1['apache_3j_bodysystem'] == 'Musculoskeletal/Skin'] = 8
test1['apache_3j_bodysystem'][test1['apache_3j_bodysystem'] == 'Metabolic'] = 9
test1['apache_3j_bodysystem'][test1['apache_3j_bodysystem'] == 'Gynecological'] = 10

In [ ]:
test1['icu_admit_source'][test1['icu_admit_source'] == 'Other ICU'] = 0
test1['icu_admit_source'][test1['icu_admit_source'] == 'Other Hospital'] = 1
test1['icu_admit_source'][test1['icu_admit_source'] == 'Floor'] = 2
test1['icu_admit_source'][test1['icu_admit_source'] == 'Accident & Emergency'] = 3
test1['icu_admit_source'][test1['icu_admit_source'] == 'Operating Room / Recovery'] = 4


In [ ]:
train1 = pd.get_dummies(train1, columns=['ethnicity', 'gender'])
test1 = pd.get_dummies(test1, columns=['ethnicity', 'gender'])

In [ ]:
test1.drop(['patient_id', 'hospital_id', 'encounter_id'], axis=1, inplace=True)

In [ ]:
train1.head()

In [ ]:
train1.info()

In [ ]:
cat_train = train[['icu_type', 'icu_stay_type', 'icu_admit_source', 'apache_2_bodysystem','apache_3j_bodysystem']]

In [ ]:
train.hospital_death.value_counts()

In [ ]:
train1.icu_type = pd.to_numeric(train1.icu_type)
train1.icu_stay_type = pd.to_numeric(train1.icu_stay_type)
train1.icu_admit_source = pd.to_numeric(train1.icu_admit_source)
train1.apache_2_bodysystem = pd.to_numeric(train1.apache_2_bodysystem)
train1.apache_3j_bodysystem = pd.to_numeric(train1.apache_3j_bodysystem)

In [ ]:
test1.icu_type = pd.to_numeric(test1.icu_type)
test1.icu_stay_type = pd.to_numeric(test1.icu_stay_type)
test1.icu_admit_source = pd.to_numeric(test1.icu_admit_source)
test1.apache_2_bodysystem = pd.to_numeric(test1.apache_2_bodysystem)
test1.apache_3j_bodysystem = pd.to_numeric(test1.apache_3j_bodysystem)

In [ ]:
train1.info()

In [ ]:
test1.isnull().sum()

In [ ]:
target = train.hospital_death

In [ ]:
train1.shape, test1.shape, target.shape

In [ ]:
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

nrm = Normalizer()
std = StandardScaler()
minmax = MinMaxScaler()
n_train = nrm.fit_transform(train1)
s_train = std.fit_transform(n_train)
m_train = minmax.fit_transform(s_train)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(m_train, target, random_state= 42)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=40, whiten=True).fit(x_train)
train_pca = pca.transform(x_train)
test_pca = pca.transform(x_test)
print(train_pca.shape)

In [ ]:
train_pca

In [ ]:
pca = PCA(n_components=3, whiten=True).fit(itest)
itest_pca = pca.transform(itest)
print(itest_pca.shape)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda =LDA(n_components=35)
X_train = lda.fit_transform(x_train, y_train)

In [ ]:
X_test = lda.transform(x_test)

In [ ]:
t1 = lda.transform(test1)

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression(normalize=True)
lr.fit(X_train, y_train)

In [ ]:
p = lr.predict(X_test)
from math import sqrt
from sklearn.metrics import mean_squared_error

print("RMSE:", sqrt(mean_squared_error(y_test, p)))
#f1_score(p, y_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(x_train, y_train)

In [ ]:
a = LR.predict(x_test)
p = LR.predict_proba(x_test)
p

In [ ]:
#p = LR.predict_proba(X_test)[]

from sklearn.metrics import roc_auc_score
roc_auc_score(a, y_test)

In [ ]:
m1 = LR.predict_proba(t1)[:, 1]

In [ ]:
output = pd.DataFrame(data = {'encounter_id':test['encounter_id'], 'hospital_death':m1})
output.to_csv(path_or_buf = '031.csv', index = False, quoting = 3)

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(base_score=0.3, learning_rate = 0.02,
                    num_boosting_rounds = 270,
                    n_estimators= 600, 
                    gamma=0.1,max_depth=10, 
                    subsample=0.8)
xgb.fit(x_train, y_train)

In [ ]:
a = xgb.predict(x_test)
from sklearn.metrics import roc_auc_score
roc_auc_score(a, y_test)

In [ ]:
xgb.predict_proba(X_test)

In [ ]:
for i in mm:
    if i 

In [ ]:
l = xgb.predict_proba(t1)[:, 1]
l

In [ ]:
#l = xgb.predict_proba(t1)[1
output = pd.DataFrame(data = {'encounter_id':test['encounter_id'], 'hospital_death':l})
output.to_csv(path_or_buf = 'b2.csv', index = False, quoting = 3)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
gr = GradientBoostingClassifier(learning_rate=0.05, n_estimators=500, random_state=42)
gr.fit(X_train, y_train)

In [ ]:
p = gr.predict(X_test)
roc_auc_score(y_test, p)

In [ ]:
print("RMSE:", sqrt(mean_squared_error(y_test, p)))

In [ ]:
pre = gr.predict(test1)
output = pd.DataFrame(data = {'encounter_id':test['encounter_id'], 'hospital_death':pre})
output.to_csv(path_or_buf = 'a2.csv', index = False, quoting = 3)

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=1000)
model.fit(X_train, y_train, verbose=100)

In [ ]:
m1=model.predict(X_test)
#print("RMSE:", sqrt(mean_squared_error(y_test, p)))

In [ ]:
roc_auc_score(y_test, m1)

In [ ]:
p = model.predict_proba(test1)[:, 0]
output = pd.DataFrame(data = {'encounter_id':test['encounter_id'], 'hospital_death':p})
output.to_csv(path_or_buf = '04.csv', index = False, quoting = 3)

In [ ]:
train1.shape

In [ ]:
a = range(0, 86)
cat_features = list(a)
print(cat_features)


from catboost import Pool
pool = Pool(data=train1, label=target)

from sklearn.model_selection import train_test_split

data = train_test_split(train1, target, test_size=0.2, random_state=0, stratify=target)
X_train, X_validation, y_train, y_validation = data

train_pool = Pool(
    data=X_train, 
    label=y_train
)

validation_pool = Pool(
    data=X_validation, 
    label=y_validation
)

In [ ]:
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03,
    custom_loss=['AUC', 'Accuracy']
)

model.fit(
    train_pool,
    eval_set=validation_pool,
    verbose=False,
    plot=True
);

In [ ]:
p = model.predict_proba(test1)[:, 0]
output = pd.DataFrame(data = {'encounter_id':test['encounter_id'], 'hospital_death':p})
output.to_csv(path_or_buf = '05.csv', index = False, quoting = 3)

In [ ]:
model.predict_proba(test1)

In [ ]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(
    iterations=2500,
    learning_rate=0.015,
    loss_function='RMSE'
)

model.fit(
    train_pool,
    eval_set=validation_pool,
    verbose=False,
    plot=True
);

In [ ]:
p = model.predict(test1)
output = pd.DataFrame(data = {'encounter_id':test['encounter_id'], 'hospital_death':p})
output.to_csv(path_or_buf = '07.csv', index = False, quoting = 3)

In [ ]:
from catboost import cv

params = {
    'loss_function': 'RMSE',
    'iterations': 2000,
    'custom_loss': 'AUC',
    'learning_rate': 0.02,
}

cv_data = cv(
    params = params,
    pool = train_pool,
    fold_count=10,
    shuffle=True,
    partition_random_seed=0,
    plot=True,
    verbose=False
)

In [ ]:
cv_data

In [ ]:
best_value = np.min(cv_data['test-Logloss-mean'])
best_iter = np.argmin(cv_data['test-Logloss-mean'])

print('Best validation Logloss score, not stratified: {:.4f}±{:.4f} on step {}'.format(
    best_value,
    cv_data['test-Logloss-std'][best_iter],
    best_iter)
)